In [5]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import random
import time
#import html5lib

In [ ]:
'''
Notes

11.22. 0:00
- 1-0 coding rappers, dropping all the R&B artists at the beginning of the process

11.22. 4:00
- corrected the URLs, but failed to overwrite the dataframe

11.22. 20:30
- df with 105 additional artists, with all good songurls


# 0.1. Blocking defense

## Delay between requests

In [6]:
delays = np.linspace(20,40,21)
#delays = np.linspace(3,10,7)
#delays = np.linspace(15,35,21)

## Rotating user agents

In [7]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

Previously I tried using rotating proxies, but free proxies from 'https://free-proxy-list.net/' or services of https://luminati.io/ resulted in azlyrics.com blocking me all the same.

# 0.2. Functions

In [8]:
def get_soup(url):
    '''
    Tries to get the html code of a web page, returns Failure if it is unsuccessful.
    '''
    # choosing a random header from the previously given list
    header = {'User-Agent' : random.choice(user_agent_list)} 
    #header = {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.112 Safari/534.30"}

    # delaying the request for a random number of seconds from the range given in delays
    delay = random.choice(delays)
    time.sleep(delay)
    
    try:
        r = requests.get(url, 
                         headers = header,
                         timeout = 15)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content,'html.parser')
            siker = True
        else:
            print('Bad status code! STOPPED')
            siker=False
            soup = ''
    except:
        soup=""
        siker=False
    
    # If it could reach the page, but got an error message, the function returns failure
    try:
        soup.find("div", {"class":"error_code"}).text==404
        siker=False
    
    except:
        pass
    
    # The function returns a boolean (success), and the html text of the webpage
    return siker,soup

def get_artist_n(url,n):
    '''
    returns the n-th artist on the Billbioard list
    UNUSED
    '''
    siker,soup = get_soup(url)
    listpiece = soup.find('article', {'class' : 'ye-chart-item', 'data-item-rank' : n})
    return listpiece.find('div', {'class' : 'ye-chart-item__title'}).text.strip()

def get_all_artists(url):
    '''
    returns a list containing the first 50 artists on the Billboard page
    '''
    artists = []
    siker,soup = get_soup(url)
    boxes = soup.find_all('article', {'class' : 'ye-chart-item'})
    for box in boxes:
        artist = box.find('div', {'class' : 'ye-chart-item__title'}).text.strip()
        artists.append(artist)
    return artists

def get_all_artists_for_year(year):
    '''
    returns the list containing the first 50 artists of the requested year
    '''
    url = "https://www.billboard.com/charts/year-end/" + str(year) + "/top-r-and-b-hip-hop-artists"
    return get_all_artists(url)

def get_all_years():
    '''
    returns the top50 artists from year 2006-2018
    in 2015 number 37 and 42 are missing, so the function corrects these values to n.a.
    '''
    all_years = {}
    all_years['rank'] = np.linspace(1,50,50)
    all_years['rank_score'] = np.linspace(50,1,50)
    for year in range(2006,2019):
        all_years[year] = get_all_artists_for_year(year)
        print("Year " + str(year) + " done!" ) # vagy "Year + & year # " done!"
    all_years[2015] = all_years[2015][:36] + ['n.a.'] + all_years[2015][36:40] + ['n.a.'] + all_years[2015][40:]
    print('Year 2015 fixed')
    return all_years

def get_success_scores(df):
    '''
    each artist gets a score according to their positions on the lists,
    then these scores get sorted in a descending order in a DataFrame
    '''
    scoredic = {'artist' : [], 'score' : []}
    for year in df.iloc[:,2:]:
        for i in range(len(df[year])):
            scoredic['artist'].append(df[year][i])
            scoredic['score'].append(df['rank_score'][i])

    return pd.DataFrame(scoredic).groupby('artist').sum().sort_values('score', ascending = False)

def get_artist_urls(url):
    '''
    creates a list with the urls of all songs of the artist with the given url
    '''
    siker,soup = get_soup(url)
    box = soup.find('div', {'id' : 'listAlbum'})
    songs = box.find_all('a')
    urls = []
    for song in songs:
        try:
            url = BASEURL + song['href'][3:]
            urls.append(url)
        except:
            continue
    return urls

def get_song_data(soup):
    '''
    collects the artist, title, release year, album and lyrics for the song from the soup to a dictionary
    '''
    song_data = {} 
    try:
        box = soup.find('div', {'class' : 'col-xs-12 col-lg-8 text-center'})
    except:
        pass
    #  szöveg
    try:
        song_data['lyrics'] = box.find_all('div')[5].text.strip()
    except:
        song_data['lyrics'] = ''
    # artist
    try:
        song_data['artist'] = ''.join(soup.find('div', {'class' : 'lyricsh'}).find('h2').text.split()[:-1])
    except:
        song_data['artist'] = ''
    # album
    try:
        song_data['album'] = box.find_all('b')[2].text
    except:
        song_data['album'] = ''
    # title
    try:
        song_data['title'] = box.find_all('b')[1].text
    except:
        song_data['title'] = ''
    # year
    try:
        song_data['year'] = int(box.find('div', {'class' : 'panel songlist-panel noprint' }).text.split('\n')[1][-5:-1])
    except:
        song_data['year'] = ''
    return song_data
    
def scrape_and_info_json(artistrange):
    '''
    takes a touple and scrapes the soups of the artists in that range for each of their songs
    and stores the song data of each song in the dataframe
    each artist will be exported as a .json and each song will be exported as a .txt
    '''
    print('This is scraping artists between ', artistrange)
    
    allsongs = []

    for artist in lfdict[artistrange[0]:artistrange[1]]:
        print('Artist ', artist['simplename'], ' started!')
        songno = 1
        for song in artist['songs']:
            url = song['url']
            siker, soup = get_soup(url)
            #runs the get_song_data function on the soup, which returns a dictionary with the song data
            song_data = get_song_data(soup) 
            song['artist'] = song_data['artist']
            song['artistscore'] = artist['score']
            song['album'] = song_data['album']
            song['title'] = song_data['title']
            song['year'] = song_data['year']
            song['lyrics'] = song_data['lyrics']
            # we create a separate list for the songs to create a DataFrame later
            allsongs.append(song) 
            print('Got a song successfully!')
            filename = '../CODE/Scrape/Songs/' + str(artist['simplename']) + '_song' + str(songno) + '.txt'
            f = open(filename, 'w', encoding = 'utf-8')
            f.write(str(song['lyrics']))
            f.close()
            songno += 1
        # writes the list of song dictionaries into a file
        filename = '../CODE/Scrape/' + str(artist['simplename']) + '.json'
        json.dump(artist, open(filename, 'w'))
        print('Successfully got all songs of ',
              artist['simplename'], ' and saved it to a json!')
    # at the end it writes the whole lfdict file to a txt
    filename = '../CODE/lfdict_out_' + str(artistrange[1]) + '.json'
    json.dump(lfdict, open(filename, 'w'))
    return allsongs
 

# 0.3. URL set

In [30]:
BBURL = "https://www.billboard.com/charts/year-end/2018/top-r-and-b-hip-hop-artists"
BASEURL = 'https://www.azlyrics.com/'

#examples for testing
artisturl = 'https://www.azlyrics.com/e/eminem.html'
songurl = 'https://www.azlyrics.com/lyrics/eminem/withoutme.html'
songurl2 = 'https://www.azlyrics.com/lyrics/kanyewest/wedontcare.html'
songurl3 = 'https://www.azlyrics.com/lyrics/lilpump/guccigang.html'

# 1. Scrape the top 50 artists for years 2006-2018

In [6]:
bb = get_all_years()
bb

Year 2006 done!
Year 2007 done!
Year 2008 done!
Year 2009 done!
Year 2010 done!
Year 2011 done!
Year 2012 done!
Year 2013 done!
Year 2014 done!
Year 2015 done!
Year 2016 done!
Year 2017 done!
Year 2018 done!
Year 2015 fixed


{'rank': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.]),
 'rank_score': array([50., 49., 48., 47., 46., 45., 44., 43., 42., 41., 40., 39., 38.,
        37., 36., 35., 34., 33., 32., 31., 30., 29., 28., 27., 26., 25.,
        24., 23., 22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12.,
        11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.]),
 2006: ['Mary J. Blige',
  'T.I.',
  'Jamie Foxx',
  'Ne-Yo',
  'Chris Brown',
  'Mariah Carey',
  'Beyonce',
  'Lil Wayne',
  'Keyshia Cole',
  'Yung Joc',
  'Ludacris',
  'Dem Franchize Boyz',
  'Lyfe Jennings',
  'Anthony Hamilton',
  'Busta Rhymes',
  'Heather Headley',
  'LeToya',
  'Sean Paul',
  'Kirk Franklin',
  'Chamillionaire',
  'Rick Ross',
  'Juelz Santana',
  'Eminem',
  'The Notorious B.I.G.

In [7]:
df = pd.DataFrame(bb)
df.head(5)

,rank,rank_score,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,1.0,50.0,Mary J. Blige,Beyonce,Alicia Keys,Beyonce,Eminem,Lil Wayne,Drake,Justin Timberlake,Beyonce,Drake,Drake,Drake,Drake
1,2.0,49.0,T.I.,Robin Thicke,Lil Wayne,Jamie Foxx,Drake,Nicki Minaj,Rihanna,Macklemore & Ryan Lewis,Pharrell Williams,The Weeknd,Beyonce,Kendrick Lamar,Post Malone
2,3.0,48.0,Jamie Foxx,JAY-Z,Keyshia Cole,Keyshia Cole,Alicia Keys,Rihanna,Nicki Minaj,Drake,Eminem,Fetty Wap,Rihanna,Bruno Mars,Cardi B
3,4.0,47.0,Ne-Yo,Ciara,Mary J. Blige,Maxwell,Usher,Chris Brown,Lil Wayne,Rihanna,Iggy Azalea,Nicki Minaj,The Weeknd,The Weeknd,XXXTENTACION
4,5.0,46.0,Chris Brown,Musiq Soulchild,Chris Brown,Kanye West,The Black Eyed Peas,Kanye West,Chris Brown,JAY-Z,John Legend,J. Cole,Future,Future,Migos


In [8]:
scores = get_success_scores(df)
scores.head(10)

,score
artist,
Drake,476.0
Beyonce,453.0
Chris Brown,453.0
Rihanna,409.0
Lil Wayne,379.0
Kanye West,345.0
JAY-Z,343.0
Eminem,327.0
Nicki Minaj,290.0


In [9]:
scores.shape

(263, 1)

There are 263 artists who made it to the Billboard top 50 in the aviable 13 years.

## Save to .csv

In [10]:
df.to_csv('billboard_lists.csv', sep=';')
scores.to_csv('artist_scores.csv', sep = ';')

## Drop the R&B artists, as they are irrelevant in this thesis

I will have to choose them by hand. Most rappers I recognize by name, but if there is an artist I don't recognize, I will use Wikipedia and the Musicbrainz database to determine the genre. This process takes place outside of this notebook, I will read the results (coded with 1-0) from a csv.

In [89]:
genres = pd.read_csv('artist_scores_genre_1.csv', sep = ';')
genres.head(5)

,artist,score,rapper
0,Drake,476.0,1.0
1,Beyonce,453.0,0.0
2,Chris Brown,453.0,1.0
3,Rihanna,409.0,0.0
4,Lil Wayne,379.0,1.0


In [90]:
# Keeping only the rappers
print('Before: ', genres.shape)
genres = genres[genres['rapper']==1]
print('After: ', genres.shape)

Before:  (263, 3)
After:  (181, 3)


In [91]:
genres = genres.reset_index(drop=True)
genres.to_csv('rapper_scores.csv', sep = ';')

# 2. Creating the leftover artist Df

They are called leftover artists, because I have already scraped the songs of the first 56 artists in a previous project (with very similar code).

Now I want to extend my database, with the rappers who appeared at least once in the Billboard charts.

## Names and estimated URLs

In [92]:
genres.head(10)

,artist,score,rapper
0,Drake,476.0,1.0
1,Chris Brown,453.0,1.0
2,Lil Wayne,379.0,1.0
3,Kanye West,345.0,1.0
4,JAY-Z,343.0,1.0
5,Eminem,327.0,1.0
6,Nicki Minaj,290.0,1.0
7,T.I.,275.0,1.0
8,Trey Songz,275.0,1.0
9,Usher,249.0,1.0


In [93]:
genres[genres['artist']=="Nelly"]

,artist,score,rapper
68,Nelly,46.0,1.0


In [174]:
leftover = genres[69:]
leftover = leftover.reset_index(drop=True)
leftover.head(5)

,artist,score,rapper
0,Fabolous,46.0,1.0
1,Kid Cudi,45.0,1.0
2,Waka Flocka Flame,45.0,1.0
3,Desiigner,45.0,1.0
4,Monica,45.0,1.0


In [175]:
leftover.artist

0               Fabolous
1               Kid Cudi
2      Waka Flocka Flame
3              Desiigner
4                 Monica
             ...        
107                  UGK
108                  D4L
109            MadeinTYO
110                6LACK
111             YelaWolf
Name: artist, Length: 112, dtype: object

In [176]:
urlpieces = []
firstletters = []

In [177]:
for name in leftover.artist:
    simplename = name.replace(' ', '')
    simplename = simplename.replace('.','')
    simplename = simplename.replace('-','')
    simplename = simplename.replace('$','')
    simplename = simplename.lower()
    firstletters.append(simplename[0])
    urlpieces.append(simplename)
    print(simplename)

fabolous
kidcudi
wakaflockaflame
desiigner
monica
jeezy
kevingates
juicewrld
silento
tyrese
demfranchizeboyz
florida
ceelogreen
drdre
richhomiequan
torylanez
6ix9ine
erykahbadu
bustarhymes
lilbaby
lloyd
tydollaign
seanpaul
youngboyneverbrokeagain
ellamai
badmeetsevil
chrisettemichele
chamillionaire
nf
twayne
nas
juelzsantana
lilpump
bowwow
aloeblacc
birdman
wanz
kentjones
chancetherapper
thenotoriousbig
lilyachty
unk
pleasurep
dram
juicyj
augustalsina
playboicarti
shawtylo
three6mafia
prettyricky
metroboomin
richthekid
raydalton
webbie
aboogiewitdahoodie
lilskies
bobbyshmurda
diddydirtymoney
macmiller
timbaland
dejloaf
thecarters
russ
sagethegemini
blackbear
blocboyjb
richboy
offset
quavo
theroots
jimjones
kyle
ilovememphis
trippieredd
jadakiss
e40
chiefkeef
yungberg
mims
youngdro
cahout
rocko
jidenna
partynextdoor
kirkobangz
gunna
lil'boosie
bigboi
ayo&teo
acehood
trinidadjames
rayj
otgenasis
davidbanner
famousdex
bobbyv
zayhilfigerrr&zayionmccall
purpleribbonallstars
machinegunkelly


In [178]:
urls

['https://www.azlyrics.com/f/fabolous.html',
 'https://www.azlyrics.com/k/kidcudi.html',
 'https://www.azlyrics.com/w/wakaflockaflame.html',
 'https://www.azlyrics.com/d/desiigner.html',
 'https://www.azlyrics.com/m/monica.html',
 'https://www.azlyrics.com/j/jeezy.html',
 'https://www.azlyrics.com/k/kevingates.html',
 'https://www.azlyrics.com/j/juicewrld.html',
 'https://www.azlyrics.com/s/silento.html',
 'https://www.azlyrics.com/t/tyrese.html',
 'https://www.azlyrics.com/d/demfranchizeboyz.html',
 'https://www.azlyrics.com/f/florida.html',
 'https://www.azlyrics.com/c/ceelogreen.html',
 'https://www.azlyrics.com/d/drdre.html',
 'https://www.azlyrics.com/r/richhomiequan.html',
 'https://www.azlyrics.com/t/torylanez.html',
 'https://www.azlyrics.com/6/6ix9ine.html',
 'https://www.azlyrics.com/e/erykahbadu.html',
 'https://www.azlyrics.com/b/bustarhymes.html',
 'https://www.azlyrics.com/l/lilbaby.html',
 'https://www.azlyrics.com/l/lloyd.html',
 'https://www.azlyrics.com/t/tydollaign.h

In [179]:
urls = []
for i in range(len(urlpieces)):
    urls.append('https://www.azlyrics.com/' + firstletters[i] + '/' + urlpieces[i] + '.html')
    
leftover['simplename'] = urlpieces
leftover['url'] = urls

leftover.head(5)

,artist,score,rapper,simplename,url
0,Fabolous,46.0,1.0,fabolous,https://www.azlyrics.com/f/fabolous.html
1,Kid Cudi,45.0,1.0,kidcudi,https://www.azlyrics.com/k/kidcudi.html
2,Waka Flocka Flame,45.0,1.0,wakaflockaflame,https://www.azlyrics.com/w/wakaflockaflame.html
3,Desiigner,45.0,1.0,desiigner,https://www.azlyrics.com/d/desiigner.html
4,Monica,45.0,1.0,monica,https://www.azlyrics.com/m/monica.html


Save to .csv

In [180]:
leftover.to_csv('leftover_artists.csv', sep=';')

In [181]:
leftover['url']

0             https://www.azlyrics.com/f/fabolous.html
1              https://www.azlyrics.com/k/kidcudi.html
2      https://www.azlyrics.com/w/wakaflockaflame.html
3            https://www.azlyrics.com/d/desiigner.html
4               https://www.azlyrics.com/m/monica.html
                            ...                       
107                https://www.azlyrics.com/u/ugk.html
108                https://www.azlyrics.com/d/d4l.html
109          https://www.azlyrics.com/m/madeintyo.html
110              https://www.azlyrics.com/6/6lack.html
111           https://www.azlyrics.com/y/yelawolf.html
Name: url, Length: 112, dtype: object

## Test these URL-s

In [104]:
# Try for one instance
artist = leftover.iloc[0]

delay = random.choice(np.linspace(15,40,26))
print(delay, ' second delay')
time.sleep(delay)

r = requests.get(artist['url'], headers = {'User-Agent' : random.choice(user_agent_list)})

if r.status_code == 200:
    artist['songs'] = []
    urls = get_artist_urls(artist['url'])
    for url in urls:
        song = {}
        song['url'] = url
        artist['songs'].append(song)  
    print(artist['simplename'], ' success!')
else:
    print(artist['simplename'], ' FAILED!')

27.0  second delay


c:\users\hp\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


fabolous  success!


In [111]:
'''
Try for all artists: collect all song URL-s in a list of dictionaries, or write the url in a list if it is wrong
'''
bads = []
for index, row in leftover.iterrows():
    
    print('Started ', row['simplename'])
    
    delay = random.choice(np.linspace(15,40,26))
    print(delay, ' second delay')
    time.sleep(delay)
    
    r = requests.get(row['url'], headers = {'User-Agent' : random.choice(user_agent_list)})
    
    if r.status_code == 200:
        row['songs'] = []
        urls = get_artist_urls(row['url'])
        for url in urls:
            songy = {}
            songy['url'] = url
            row['songs'].append(songy)  
        print(row['simplename'], ' success!')
    else:
        print(row['simplename'], ' FAILED!')
        bads.append(row['url'])
        continue

Started  fabolous
17.0  second delay
fabolous  success!
Started  kidcudi
28.0  second delay
kidcudi  success!
Started  wakaflockaflame
22.0  second delay
wakaflockaflame  success!
Started  desiigner
31.0  second delay
desiigner  success!
Started  monica
35.0  second delay
monica  success!
Started  jeezy
17.0  second delay
jeezy  FAILED!
Started  kevingates
36.0  second delay
kevingates  success!
Started  juicewrld
38.0  second delay
juicewrld  success!
Started  silento
27.0  second delay
silento  success!
Started  tyrese
23.0  second delay
tyrese  success!
Started  demfranchizeboyz
38.0  second delay
demfranchizeboyz  success!
Started  florida
37.0  second delay
florida  success!
Started  ceelogreen
40.0  second delay
ceelogreen  success!
Started  drdre
16.0  second delay
drdre  success!
Started  richhomiequan
18.0  second delay
richhomiequan  success!
Started  torylanez
31.0  second delay
torylanez  success!
Started  6ix9ine
37.0  second delay
6ix9ine  FAILED!
Started  erykahbadu
40.0

In [114]:
# which URLs were not good?
bads

['https://www.azlyrics.com/j/jeezy.html',
 'https://www.azlyrics.com/6/6ix9ine.html',
 'https://www.azlyrics.com/e/erykahbadu.html',
 'https://www.azlyrics.com/b/bustarhymes.html',
 'https://www.azlyrics.com/t/tydollaign.html',
 'https://www.azlyrics.com/s/seanpaul.html',
 'https://www.azlyrics.com/y/youngboyneverbrokeagain.html',
 'https://www.azlyrics.com/j/juelzsantana.html',
 'https://www.azlyrics.com/w/wanz.html',
 'https://www.azlyrics.com/t/thenotoriousbig.html',
 'https://www.azlyrics.com/p/playboicarti.html',
 'https://www.azlyrics.com/r/raydalton.html',
 'https://www.azlyrics.com/a/aboogiewitdahoodie.html',
 'https://www.azlyrics.com/t/thecarters.html',
 'https://www.azlyrics.com/t/theroots.html',
 'https://www.azlyrics.com/i/ilovememphis.html',
 "https://www.azlyrics.com/l/lil'boosie.html",
 'https://www.azlyrics.com/a/ayo&teo.html',
 'https://www.azlyrics.com/d/davidbanner.html',
 'https://www.azlyrics.com/f/famousdex.html',
 'https://www.azlyrics.com/z/zayhilfigerrr&zayion

In [115]:
# Save bads for checkpoint
f2 = open('bads.txt', 'w')
savestring2 = str(bads)
f2.write(savestring2)
f2.close()

## Reading it in

In [134]:
f4 = open('bads.txt', 'r')
readtext = f4.read()
bads = eval(readtext)
bads

['https://www.azlyrics.com/j/jeezy.html',
 'https://www.azlyrics.com/6/6ix9ine.html',
 'https://www.azlyrics.com/e/erykahbadu.html',
 'https://www.azlyrics.com/b/bustarhymes.html',
 'https://www.azlyrics.com/t/tydollaign.html',
 'https://www.azlyrics.com/s/seanpaul.html',
 'https://www.azlyrics.com/y/youngboyneverbrokeagain.html',
 'https://www.azlyrics.com/j/juelzsantana.html',
 'https://www.azlyrics.com/w/wanz.html',
 'https://www.azlyrics.com/t/thenotoriousbig.html',
 'https://www.azlyrics.com/p/playboicarti.html',
 'https://www.azlyrics.com/r/raydalton.html',
 'https://www.azlyrics.com/a/aboogiewitdahoodie.html',
 'https://www.azlyrics.com/t/thecarters.html',
 'https://www.azlyrics.com/t/theroots.html',
 'https://www.azlyrics.com/i/ilovememphis.html',
 "https://www.azlyrics.com/l/lil'boosie.html",
 'https://www.azlyrics.com/a/ayo&teo.html',
 'https://www.azlyrics.com/d/davidbanner.html',
 'https://www.azlyrics.com/f/famousdex.html',
 'https://www.azlyrics.com/z/zayhilfigerrr&zayion

## Correcting the bad URL-s by hand

#### Eventually decided to do this part outside of code, corrected the URLs in the leftover_artists.csv file, creating leftover_artists_corrected.csv

In [144]:
badnums = [5, 16, 17, 18, 21, 22, 23, 31, 39, 46, 54, 69, 72, 86, 88, 93, 94, 102, 110]

In [145]:
fixed_urls = [
    'https://www.azlyrics.com/y/youngjeezy.html',
    'https://www.azlyrics.com/19/6ix9ine.html',
    'https://www.azlyrics.com/b/badu.html',
    'https://www.azlyrics.com/b/busta.html',
    'https://www.azlyrics.com/t/tydollasign.html',
    'https://www.azlyrics.com/p/paul.html',
    'https://www.azlyrics.com/n/nbayoungboy.html',
    'https://www.azlyrics.com/s/santanaj.html',
    'https://www.azlyrics.com/n/notorious.html',
    'https://www.azlyrics.com/p/playboi-carti.html',
    'https://www.azlyrics.com/b/boogiewitdahoodie.html',
    'https://www.azlyrics.com/r/roots.html',
    'https://www.azlyrics.com/i/iheartmemphis.html',
    'https://www.azlyrics.com/l/lilboosie.html',
    'https://www.azlyrics.com/a/ayoteo.html',
    'https://www.azlyrics.com/b/banner.html',
    'https://www.azlyrics.com/f/famous-dex.html',
    'https://www.azlyrics.com/b/beastie.html',
    'https://www.azlyrics.com/19/6lack.html'
]

In [149]:
# Does't work properly, didn't use
for i in range(len(badnums)):
    leftover.iloc[badnums[i]]['url'] = fixed_urls[i]
    print(leftover.iloc[badnums[i]]['url'])

https://www.azlyrics.com/j/jeezy.html
https://www.azlyrics.com/6/6ix9ine.html
https://www.azlyrics.com/e/erykahbadu.html
https://www.azlyrics.com/b/bustarhymes.html
https://www.azlyrics.com/t/tydollaign.html
https://www.azlyrics.com/s/seanpaul.html
https://www.azlyrics.com/y/youngboyneverbrokeagain.html
https://www.azlyrics.com/j/juelzsantana.html
https://www.azlyrics.com/t/thenotoriousbig.html
https://www.azlyrics.com/p/playboicarti.html
https://www.azlyrics.com/a/aboogiewitdahoodie.html
https://www.azlyrics.com/t/theroots.html
https://www.azlyrics.com/i/ilovememphis.html
https://www.azlyrics.com/l/lil'boosie.html
https://www.azlyrics.com/a/ayo&teo.html
https://www.azlyrics.com/d/davidbanner.html
https://www.azlyrics.com/f/famousdex.html
https://www.azlyrics.com/b/beastieboys.html
https://www.azlyrics.com/6/6lack.html


c:\users\hp\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Read the corrected leftover df

In [233]:
leftover = pd.read_csv('leftover_artists_corrected.csv', sep = ';')
leftover.head(10)

,Unnamed: 0,artist,score,rapper,simplename,url
0,0,Fabolous,46.0,1.0,fabolous,https://www.azlyrics.com/f/fabolous.html
1,1,Kid Cudi,45.0,1.0,kidcudi,https://www.azlyrics.com/k/kidcudi.html
2,2,Waka Flocka Flame,45.0,1.0,wakaflockaflame,https://www.azlyrics.com/w/wakaflockaflame.html
3,3,Desiigner,45.0,1.0,desiigner,https://www.azlyrics.com/d/desiigner.html
4,4,Monica,45.0,1.0,monica,https://www.azlyrics.com/m/monica.html
5,5,Jeezy,44.0,1.0,jeezy,https://www.azlyrics.com/y/youngjeezy.html
6,6,Kevin Gates,43.0,1.0,kevingates,https://www.azlyrics.com/k/kevingates.html
7,7,Juice WRLD,43.0,1.0,juicewrld,https://www.azlyrics.com/j/juicewrld.html
8,8,Silento,42.0,1.0,silento,https://www.azlyrics.com/s/silento.html
9,9,Tyrese,41.0,1.0,tyrese,https://www.azlyrics.com/t/tyrese.html


#### Drop a couple of artists

Wanz, Rey Dalton, The Carters, Zay Hilfigerrr, Purple Ribbon All Stars, iHeartMemphis, Ayo & teo
<br> <br> 
id: 36, 52, 61, 72, 88, 96, 97

In [234]:
leftover.drop(index = [36, 52, 61, 72, 88, 96, 97], columns = 'Unnamed: 0', inplace = True)
leftover.reset_index(drop = True, inplace = True)
leftover.shape

(105, 5)

In [235]:
leftover.head(3)

,artist,score,rapper,simplename,url
0,Fabolous,46.0,1.0,fabolous,https://www.azlyrics.com/f/fabolous.html
1,Kid Cudi,45.0,1.0,kidcudi,https://www.azlyrics.com/k/kidcudi.html
2,Waka Flocka Flame,45.0,1.0,wakaflockaflame,https://www.azlyrics.com/w/wakaflockaflame.html


## Getting the song urls for each artist

In [236]:
'''
Try for all artists: collect all song URL-s in a list of dictionaries, or write the url in a list if it is wrong
'''
bads = []
songs = [] # This will be the new column of the dataframe, with a list of the songs of each artist
for index, artist in leftover.iterrows():
    artistsongs = []
    print('Started ', artist['simplename'])
    
    delay = random.choice(np.linspace(15,40,26))
    print(delay, ' second delay')
    time.sleep(delay)
    
    r = requests.get(artist['url'], headers = {'User-Agent' : random.choice(user_agent_list)})
    
    if r.status_code == 200:
        
        urls = get_artist_urls(artist['url'])
        for url in urls:
            songy = {}
            songy['url'] = url
            artistsongs.append(songy)
        songs.append(artistsongs)
        print(artist['simplename'], ' success!')
    else:
        print(artist['simplename'], ' FAILED!')
        bads.append(row['url'])
        continue

Started  fabolous
25.0  second delay
fabolous  success!
Started  kidcudi
36.0  second delay
kidcudi  success!
Started  wakaflockaflame
37.0  second delay
wakaflockaflame  success!
Started  desiigner
29.0  second delay
desiigner  success!
Started  monica
31.0  second delay
monica  success!
Started  jeezy
26.0  second delay
jeezy  success!
Started  kevingates
40.0  second delay
kevingates  success!
Started  juicewrld
23.0  second delay
juicewrld  success!
Started  silento
25.0  second delay
silento  success!
Started  tyrese
34.0  second delay
tyrese  success!
Started  demfranchizeboyz
34.0  second delay
demfranchizeboyz  success!
Started  florida
29.0  second delay
florida  success!
Started  ceelogreen
32.0  second delay
ceelogreen  success!
Started  drdre
29.0  second delay
drdre  success!
Started  richhomiequan
18.0  second delay
richhomiequan  success!
Started  torylanez
15.0  second delay
torylanez  success!
Started  6ix9ine
40.0  second delay
6ix9ine  success!
Started  erykahbadu
33

In [237]:
bads

[]

In [238]:
len(songs)

105

In [241]:
leftover['songs'] = songs
leftover.head(5)

,artist,score,rapper,simplename,url,songs
0,Fabolous,46.0,1.0,fabolous,https://www.azlyrics.com/f/fabolous.html,[{'url': 'https://www.azlyrics.com/lyrics/fabo...
1,Kid Cudi,45.0,1.0,kidcudi,https://www.azlyrics.com/k/kidcudi.html,[{'url': 'https://www.azlyrics.com/lyrics/kidc...
2,Waka Flocka Flame,45.0,1.0,wakaflockaflame,https://www.azlyrics.com/w/wakaflockaflame.html,[{'url': 'https://www.azlyrics.com/lyrics/waka...
3,Desiigner,45.0,1.0,desiigner,https://www.azlyrics.com/d/desiigner.html,[{'url': 'https://www.azlyrics.com/lyrics/desi...
4,Monica,45.0,1.0,monica,https://www.azlyrics.com/m/monica.html,[{'url': 'https://www.azlyrics.com/lyrics/moni...


In [242]:
''' Success!!! '''
leftover.to_csv('leftover_artists_with_urls.csv', sep=';')

# 3. Start scraping songs

In [9]:
leftover = pd.read_csv('../CODE/leftover_artists_with_urls.csv', sep = ';')
leftover.drop(columns = 'Unnamed: 0', inplace = True)
leftover.head(5)

,artist,score,rapper,simplename,url,songs
0,Fabolous,46.0,1.0,fabolous,https://www.azlyrics.com/f/fabolous.html,[{'url': 'https://www.azlyrics.com/lyrics/fabo...
1,Kid Cudi,45.0,1.0,kidcudi,https://www.azlyrics.com/k/kidcudi.html,[{'url': 'https://www.azlyrics.com/lyrics/kidc...
2,Waka Flocka Flame,45.0,1.0,wakaflockaflame,https://www.azlyrics.com/w/wakaflockaflame.html,[{'url': 'https://www.azlyrics.com/lyrics/waka...
3,Desiigner,45.0,1.0,desiigner,https://www.azlyrics.com/d/desiigner.html,[{'url': 'https://www.azlyrics.com/lyrics/desi...
4,Monica,45.0,1.0,monica,https://www.azlyrics.com/m/monica.html,[{'url': 'https://www.azlyrics.com/lyrics/moni...


In [10]:
lfdict = leftover.to_dict(orient = 'records')

In [11]:
# Beause the lists of songs are strings
for artist in lfdict:
    artist['songs'] = eval(artist['songs'])

In [11]:
lfdict[0]['songs']

[{'url': 'https://www.azlyrics.com/lyrics/fabolous/clickspark.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/keepinitgangsta.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/youngn.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/getright.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/rideforthis.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/oneday.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/tradeitall.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/rightnowlateron.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/takeyouhome.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/getsmart.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/cantdenyit.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/mabeeasy.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/wedontgivea.html'},
 {'url': 'https://www.azlyrics.com/lyrics/fabolous/thebadguy.html'},
 {'url': 'https://www.

## Scope of scraping

In [8]:
count = 0
for artist in lfdict:
    #print(artist['name'])
    count += len(artist['songs'])
print('The number of songs we need to scrape: ', count)

The number of songs we need to scrape:  12755


## Scraping

In [15]:
scrape_and_info_json((40,50))

This is scraping artists between  (40, 50)
Artist  unk  started!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Successfully got all songs of  unk  and saved it to a json!
Artist  pleasurep  started!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfull

Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!


Got a song successfully!
Got a song successfully!
Bad status code! STOPPED
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Bad status code! STOPPED
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Successfully got all songs of  playboicarti  and saved it to a json!
Artist  shawtylo  started!
Got a song successfully!
Got a song successfully!
Got 

Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Successfully got all songs of  three6mafia  and saved it to a json!
Artist  prettyricky  started!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Go

[{'url': 'https://www.azlyrics.com/lyrics/unk/introbeatndownyoblock.html',
  'artist': 'Unk',
  'artistscore': 26.0,
  'album': '"Beat\'n Down Yo Block!"',
  'title': '"Intro (Beat\'n Down Yo Block!)"',
  'year': 2006,
  'lyrics': '[Jelly:]\nWelfare check one, welfare check two\nThat\'s right we gon\' give it to \'em\nThis yo\' favorite DJ\'s DJ, DJ Jelly\nAnd I got me a new member of the Oomp Camp\nDJ Unk (DJ Unk hoe!) But I ain\'t gon\' talk no more\nDJ Montae, Oomp Camp this shit, niggahhhhhh!\n\n[scratched repeatedly:] "DJ Unk hoe"\n\n[Unk:]\nYo\nYo, yo\nYo\nYo, yo\nSince when the last time, you got your money\'s worth\nD-boys in two songs, niggaz I got that Nelly curse\nSo dig down in your purse, the wait no longer here\nThe type of niggaz, type of killers bootleggers fear\nNow they may rock the party, see me I\'m off the yam\nThey bought a dub or two, I\'m like a thousand grams\nMy pistol in my hand, I\'m confiscatin rocks-es\nNo SoundScans nigga I\'ma see my pockets\nSo lay it d

#### Something's wrong

In [19]:
songurl

'https://www.azlyrics.com/lyrics/eminem/withoutme.html'

In [22]:
siker, soup = get_soup(songurl)

In [24]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Eminem &quot;Without Me&quot;: Obie Trice Real name no gimmicks Two trailer-park girls go 'round the outside, 'round the outside,..." name="description"/>
<meta content="Without Me lyrics, Eminem Without Me lyrics, Eminem lyrics" name="keywords"/>
<meta content="noarchive" name="robots"/>
<meta content="//www.azlyrics.com/az_logo_tr.png" property="og:image"/>
<title>Eminem - Without Me Lyrics | AZLyrics.com</title>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css" rel="stylesheet"/>
<link href="//www.azlyrics.com/bsaz.css?1" rel="stylesheet"/>
<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 9]>
<script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script>
<script src="https://oss.ma

In [25]:
box = soup.find('div', {'class' : 'col-xs-12 col-lg-8 text-center'})
box

<div class="col-xs-12 col-lg-8 text-center">
<div class="div-share noprint">
<div class="addthis_toolbox addthis_default_style">
<a class="btn btn-xs btn-share addthis_button_facebook" href="https://www.facebook.com/sharer.php?u=https%3A%2F%2Fwww.azlyrics.com%2Flyrics%2Feminem%2Fwithoutme.html&amp;title=Eminem%20-%20Without%20Me%20Lyrics%20%7C%20AZLyrics.com&amp;display=page" target="_blank">
<span class="playblk"><img alt="Share on Facebook" class="playblk" height="25" src="//www.azlyrics.com/images/share-icons/facebook.svg" title="Share on Facebook" width="25"/></span>
</a>
<a class="btn btn-xs btn-share addthis_button_twitter" href="https://twitter.com/intent/tweet?url=https%3A%2F%2Fwww.azlyrics.com%2Flyrics%2Feminem%2Fwithoutme.html&amp;text=Eminem%20-%20Without%20Me%20Lyrics%20%7C%20AZLyrics.com" target="_blank">
<span class="playblk"><img alt="Share on Twiiter" class="playblk" height="25" src="//www.azlyrics.com/images/share-icons/twitter.svg" title="Share on Twiiter" width="25"/

In [26]:
box.find_all('div')[6].text.strip()

'if  ( /Android|webOS|iPhone|iPod|iPad|BlackBerry|IEMobile|Opera Mini/i.test(navigator.userAgent) ) \r\n  {\r\n     document.write(\'<div style="margin-left: auto; margin-right: auto;">\'+\r\n  \'<iframe scrolling="no" style="border: 0px none; overflow:hidden;" src="//adv.mxmcdn.net/br/t1.0/m_js/e_0/sn_0/l_15728268/su_0/rs_0/tr_3vUCAMhs4WfLER1H1WTB6R0XnShSrZM6cCPBX6K0wvIJx50obYztDc7n0DO2ezP8C0YminRb1JKyyDV1EpRx93GMlfV6iCOkFfhQrr_JBpFIdELHGoLwBDcRFfOMEDqC3qESx6oiqxKNCHrUpFAAcrEanfdwdDizsY61Y61BNcH1lyNM5N3UVqUA2nfmuE7RvxFazvEbt7gfgkyjG9L9T-1vDVbiv636IFKMjZrpyzv079NsqddKMiKxGJfLP-bJOKY17PVFh0bV_051dRnU53RvQaGWrKMMhD0n5znEQ1PvpThQhz2g666um2Rb5HY2Hnr4FuZ6SOM26kyjNShSfllxyPcaI4ErXFVdUXLWt8LBgYVtwBH3hf-4J3pNJurcKBidp_JvgjlxKGMcL6piGsTSM5cBWKm1gCcBNcet_Q1uOpqXhCeIZ8pB6w/" width="290px" height="50px"></iframe>\'+\r\n  \'</div>\');\r\n   }'

In [35]:
box.find_all('div')[5].text.strip()

'[Obie Trice:]\nObie Trice\nReal name no gimmicks\n\n[Eminem:]\nTwo trailer-park girls go \'round the outside, \'round the outside, \'round the outside\nTwo trailer-park girls go \'round the outside, \'round the outside, \'round the outside\n\nGuess who\'s back, back again\nShady\'s back, tell a friend\nGuess who\'s back\nGuess who\'s back\nGuess who\'s back\nGuess who\'s back\nGuess who\'s back\nGuess who\'s back\nGuess who\'s back\n\nI\'ve created a monster\n\'Cause nobody wants to see Marshall no more\nThey want Shady I\'m chopped liver\nWell if you want Shady, this is what I\'ll give ya\nA little bit of weed mixed with some hard liquor\nSome vodka that\'ll jump-start my heart\nQuicker than a shock\nWhen I get shocked at the hospital by the doctor\nWhen I\'m not cooperating\nWhen I\'m rocking the table while he\'s operating (hey!)\n\nYou waited this long now stop debating\n\'Cause I\'m back I\'m on the rag and ovulating\nI know that you got a job Ms. Cheney\nBut your husband\'s hear

In [23]:
get_song_data(soup)

{'lyrics': 'if  ( /Android|webOS|iPhone|iPod|iPad|BlackBerry|IEMobile|Opera Mini/i.test(navigator.userAgent) ) \r\n  {\r\n     document.write(\'<div style="margin-left: auto; margin-right: auto;">\'+\r\n  \'<iframe scrolling="no" style="border: 0px none; overflow:hidden;" src="//adv.mxmcdn.net/br/t1.0/m_js/e_0/sn_0/l_15728268/su_0/rs_0/tr_3vUCAMhs4WfLER1H1WTB6R0XnShSrZM6cCPBX6K0wvIJx50obYztDc7n0DO2ezP8C0YminRb1JKyyDV1EpRx93GMlfV6iCOkFfhQrr_JBpFIdELHGoLwBDcRFfOMEDqC3qESx6oiqxKNCHrUpFAAcrEanfdwdDizsY61Y61BNcH1lyNM5N3UVqUA2nfmuE7RvxFazvEbt7gfgkyjG9L9T-1vDVbiv636IFKMjZrpyzv079NsqddKMiKxGJfLP-bJOKY17PVFh0bV_051dRnU53RvQaGWrKMMhD0n5znEQ1PvpThQhz2g666um2Rb5HY2Hnr4FuZ6SOM26kyjNShSfllxyPcaI4ErXFVdUXLWt8LBgYVtwBH3hf-4J3pNJurcKBidp_JvgjlxKGMcL6piGsTSM5cBWKm1gCcBNcet_Q1uOpqXhCeIZ8pB6w/" width="290px" height="50px"></iframe>\'+\r\n  \'</div>\');\r\n   }',
 'artist': 'Eminem',
 'album': '"The Eminem Show"',
 'title': '"Without Me"',
 'year': 2002}

# 4. RESULT

Scraping created individual .json files of each scraped artists, containing a dictionary. The structure is the following:

`[ { 'name' : the name of the artist,                                  
     'score': the success score of the artist,                         
     'url'  : the url containing all the songs of the artist,          
     'songs': [ {                                                      
                 'url' : the url of the song,                          
                 'soup': the soup of the song,                         
                 'artist' : the creator of the song,                   
                 'year'   : the year the song was released,            
                 'album'  : the album the song was released on,        
                 'title'  : the title of the song,                     
                 'lyrics' : the lyrics of the song                     
                },                                                     
                {                                                      
                 song2                                                 
                }                                                      
                ...                                                    
              ]                                                        
   },                                                                  
   {                                                                   
   artist2                                                             
   },                                                                  
   ...                                                                 
 ]                                                                     `
 
 We also save each individual song in a txt.
 See an example below:

In [7]:
drake = json.load(open('../Rapper_project/JSON-ok/0-31/drake.json'))
drake

{'name': 'drake',
 'score': 476,
 'songs': [{'album': '"Room For Improvement"',
   'artist': 'Drake',
   'artistscore': 476,
   'lyrics': 'Yo what\'s goin on, this is Drake\nAnd I\'ma let you know what you about to witness aight?\nThis right here, is a Drake, and DJ Smallz collaboration\nSo I\'m from Canada, my mans from down South\nYou understand the #1 DJ in the South to be exact\nYou heard that at the VMA\'s, you heard it wherever he goes\nMy man Smallz is out there down South\nSame time reppin for Toronto, Canada y\'knahmean?\nSo this right here what you \'bout what you listenin to right NOW\nis the official, "Southern Smoke: Special Edition"\nI call it the "Room For Improvement" mixtape\nCause I\'m not perfect, and I bet neither are you if you listenin\nso you need to just accept, what\'s THERE, y\'knahmean? But look\nYou need to just let this play out\nfrom track one, to track whatever however many I put on here\nYou need to just listen to this because look man\nit\'s the first t

# 5. Scraping the songs of the 'GOAT' Artists

Source: first 25 artists from https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time?page=2

We already have scraped:
    Eminem, Kendrick Lamar, Snoop Dogg, Jaz-Z, 50 Cent, Kanye West, J. Cole, Lil Wayne, Ludacris, T.I., A$AP Rocky
    
Andre 3000 was swapped to Outkast, as most of his work was performed as part of the duo. (1 solo album vs. 6 Outkast albums). 
    
We will add them later to the dataframe from the already existing files

## Setting the URLs of the GOAT artists

In [21]:
goatsinput = [ {'name': '2pac',            'artisturl' : 'https://www.azlyrics.com/19/2pac.html'},
               {'name': 'Notorius B.I.G.', 'artisturl' : 'https://www.azlyrics.com/n/notorious.html'},
               {'name': 'Nas' ,            'artisturl' : 'https://www.azlyrics.com/n/nas.html'},
               {'name': 'Dr. Dre',         'artisturl' : 'https://www.azlyrics.com/d/drdre.html'},
               {'name': 'Ice Cube',        'artisturl' : 'https://www.azlyrics.com/i/icecube.html'},
               {'name': 'Eazy-E',          'artisturl' : 'https://www.azlyrics.com/e/eazye.html'},
               {'name': 'Outkast',         'artisturl' : 'https://www.azlyrics.com/o/outkast.html'},
               {'name': 'Rakim',           'artisturl' : 'https://www.azlyrics.com/r/rakim.html'},
               {'name': 'DMX',             'artisturl' : 'https://www.azlyrics.com/d/dmx.html'},
               {'name': 'Busta Rhymes',    'artisturl' : 'https://www.azlyrics.com/b/busta.html'},
               {'name': 'Method Man',      'artisturl' : 'https://www.azlyrics.com/m/methodman.html'},
               {'name': 'LL Cool J',       'artisturl' : 'https://www.azlyrics.com/l/llcoolj.html'},
               {'name': 'Mos Def',         'artisturl' : 'https://www.azlyrics.com/m/mosdef.html'},
               {'name': 'Ghostface Killah','artisturl' : 'https://www.azlyrics.com/g/ghostface.html'}
             ]
                

In [11]:
print('We have 11 artists already from the top 25 of all time, we still have to scrape ', len(goatsinput), ' artists.')

We have 11 artists already from the top 25 of all time, we still have to scrape  14  artists.


In [17]:
goatsinput2 = [{'name': 'Nate Dogg',       'artisturl' : 'https://www.azlyrics.com/n/natedogg.html'},
               {'name': 'Joyner Lucas', 'artisturl' : 'https://www.azlyrics.com/j/joynerlucas.html'},
               {'name': 'Big L' ,            'artisturl' : 'https://www.azlyrics.com/b/bigl.html'},
               {'name': 'Tech N9ne',         'artisturl' : 'https://www.azlyrics.com/t/techn9ne.html'},
               {'name': 'Royce da 59',        'artisturl' : 'https://www.azlyrics.com/r/royceda59.html'},
               {'name': 'Redman',          'artisturl' : 'https://www.azlyrics.com/r/redman.html'},
               {'name': 'Lauryn Hill',         'artisturl' : 'https://www.azlyrics.com/h/hill.html'},
               {'name': 'Common',           'artisturl' : 'https://www.azlyrics.com/c/common.html'},
               {'name': 'Pusha T',             'artisturl' : 'https://www.azlyrics.com/p/pushat.html'},
               {'name': 'Tyler the Creator',    'artisturl' : 'https://www.azlyrics.com/t/tylerthecreator.html'},
               {'name': 'Joey Badass',      'artisturl' : 'https://www.azlyrics.com/j/joeybada.html'},
               {'name': 'Big Pun',       'artisturl' : 'https://www.azlyrics.com/b/bigpun.html'},
               {'name': 'KRS-One',         'artisturl' : 'https://www.azlyrics.com/k/krsone.html'},
             ]
                

To have 50 GOAT artists, we need 13 more.

## Getting the songurls for each GOAT artist

In [32]:
for artist in goatsinput2:
    artist['songurls']= get_artist_urls(artist['artisturl'])
    print(artist['name'], ' done!')

Nate Dogg  done!
Joyner Lucas  done!
Big L  done!
Tech N9ne  done!
Royce da 59  done!
Redman  done!
Lauryn Hill  done!
Common  done!
Pusha T  done!
Tyler the Creator  done!
Joey Badass  done!
Big Pun  done!
KRS-One  done!


In [ ]:
def get_artist_urls(url):
    '''
    creates a list with the urls of all songs of the artist with the given url
    '''
    siker,soup = get_soup(url)
    box = soup.find('div', {'id' : 'listAlbum'})
    songs = box.find_all('a')
    urls = []
    for song in songs:
        try:
            url = BASEURL + song['href'][3:]
            urls.append(url)
        except:
            continue
    return urls

In [23]:
siker, soup = get_soup('https://www.azlyrics.com/n/natedogg.html')

In [29]:
soup.find('div', {'id' : 'listAlbum'}).find_all('a')[0]['href'][3:]

'lyrics/natedogg/hardestmanintown.html'

In [33]:
c = 0
for artist in goatsinput2:
    print(artist['name'], ' has ', len(artist['songurls']), ' songs.')
    c += len(artist['songurls'])
print(c, 'songs altogether.')

Nate Dogg  has  61  songs.
Joyner Lucas  has  64  songs.
Big L  has  61  songs.
Tech N9ne  has  575  songs.
Royce da 59  has  185  songs.
Redman  has  166  songs.
Lauryn Hill  has  43  songs.
Common  has  172  songs.
Pusha T  has  81  songs.
Tyler the Creator  has  125  songs.
Joey Badass  has  102  songs.
Big Pun  has  57  songs.
KRS-One  has  208  songs.
1900 songs altogether.


## Scraping of these artists

In [38]:
for artist in goatsinput2:
    print('Artist ', artist['name'], ' started!')
    artist['songs'] = []
    songno = 1
    for url in artist['songurls']:
        siker, soup = get_soup(url)
        #runs the get_song_data function on the soup, which returns a dictionary with the song data
        song_data = get_song_data(soup) 
        song = {}
        song['artist'] = song_data['artist']
        song['album'] = song_data['album']
        song['title'] = song_data['title']
        song['year'] = song_data['year']
        song['lyrics'] = song_data['lyrics']
        # we add the song data to the artists songs
        artist['songs'].append(song) 
        print('Got a song successfully!')
        filename = '../CODE/Scrape/Songs/' + str(artist['name']) + '_song' + str(songno) + '.txt'
        f = open(filename, 'w', encoding = 'utf-8')
        f.write(str(song['lyrics']))
        f.close()
        songno += 1
    # writes the list of song dictionaries into a file
    filename = '../CODE/Scrape/' + str(artist['name']) + '.json'
    json.dump(artist, open(filename, 'w'))
    print('Successfully got all songs of ',
          artist['name'], ' and saved it to a json!')

Artist  Nate Dogg  started!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfull

Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!


Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Bad status code! STOPPED
Got a song successfully!
Got a song successfully!
Got a song successfully!
Bad status code! STOPPED
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!


Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!
Got a song successfully!


KeyboardInterrupt: 

In [36]:
for artist in goatsinput2:
    print(artist['name'])

Nate Dogg
Joyner Lucas
Big L
Tech N9ne
Royce da 59
Redman
Lauryn Hill
Common
Pusha T
Tyler the Creator
Joey Badass
Big Pun
KRS-One
